# YOLOplan Pipeline with Centralized Configuration

This notebook provides a complete and easy-to-manage pipeline to train a YOLOplan model. All major settings are controlled from the **Master Configuration** cell below.

## 1. Environment Setup

This first step clones the YOLOplan repository, installs all dependencies, and verifies the GPU. You only need to run this section once per session.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clone the YOLOplan repository and navigate into the directory
import os
if not os.path.exists('YOLOplan'):
    !git clone https://github.com/DynMEP/YOLOplan.git
    os.chdir('YOLOplan')
else:
    os.chdir('YOLOplan')

# 1. Install standard dependencies
!pip install -r requirements.txt roboflow -q

# 2. FORCE UPDATE Ultralytics (CRITICAL for YOLO11 support)
!pip install -U ultralytics

# Verify GPU and Version
import ultralytics
print(f"Ultralytics Version: {ultralytics.__version__} (Should be 8.3.0 or higher for YOLO11)")
!nvidia-smi

Cloning into 'YOLOplan'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 70 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 63.47 KiB | 9.07 MiB/s, done.
Resolving deltas: 100% (21/21), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 114.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.

## 2. Master Configuration

**Action Required:** Configure your entire pipeline here. Update your Roboflow details, choose your model, and decide whether to enable or disable augmentations. The settings you define in this cell will be used by all subsequent steps.

In [3]:
#@title ⚙️ Master Configuration Panel (Final Version)
#@markdown ---
#@markdown ### 🔑 Roboflow Project Details
ROBOFLOW_WORKSPACE_ID = "elliotttmiller"  #@param {type:"string"}
ROBOFLOW_PROJECT_ID = "hvacai-s3kda"      #@param {type:"string"}
ROBOFLOW_VERSIONS = [24, 27, 28]          #@param {type:"raw"}

#@markdown ---
#@markdown ### 💾 Storage & Naming
DRIVE_ROOT = "/content/drive/MyDrive/hvac_training" #@param {type:"string"}
RUN_NAME = "yolo11m_run_v8_TEXT_FINETUNE"   #@param {type:"string"}

#@markdown ---
#@markdown ### 🧠 Progressive Learning
USE_PREVIOUS_WEIGHTS = True     #@param {type:"boolean"}
PREVIOUS_WEIGHTS_PATH = "/content/drive/MyDrive/hvac_training/yolo11m_run_v8/weights/best.pt" #@param {type:"string"}

#@markdown ---
#@markdown ### ⚖️ Dataset Splitting & Curation
TRAIN_RATIO = 0.8   #@param {type:"number"}
VAL_RATIO   = 0.15  #@param {type:"number"}
TEST_RATIO  = 0.05  #@param {type:"number"}
REQUIRE_SPATIAL_CONTAINMENT = True     #@param {type:"boolean"}
CONTAINER_CLASSES = ["instrument_shared_primary", "instrument_discrete_primary", "instrument_discrete_field", "instrument_discrete_aux"] #@param {type:"raw"}
REQUIRED_CONTENTS = ["id_letters", "tag_number"] #@param {type:"raw"}

#@markdown ---
#@markdown ### 🚀 Training Settings
MODEL = 'yolo11m.pt'
EPOCHS = 20
IMG_SIZE = 1280
BATCH_SIZE = 4
CLS_GAIN = 1.0
ENABLE_AUGMENTATIONS = False

#@markdown ---
#@markdown ### 📊 Live Monitoring Suite
LOGGER = "TensorBoard" #@param ["TensorBoard", "Weights & Biases"]

In [ ]:
#@title ✅ Pre-flight Configuration Check
import os

print("--- Checking Required Variables for Training ---")
required_vars = [
    'DRIVE_ROOT', 'RUN_NAME', 'MODEL', 'USE_PREVIOUS_WEIGHTS',
    'PREVIOUS_WEIGHTS_PATH', 'EPOCHS', 'IMG_SIZE', 'BATCH_SIZE',
    'ENABLE_AUGMENTATIONS'
]

all_vars_exist = True
for var in required_vars:
    if var not in locals():
        print(f"❌ FAILED: The variable '{var}' is missing from memory.")
        all_vars_exist = False
    else:
        # Optional: Print the value to confirm it's correct
        # print(f"  - {var}: {locals()[var]}")
        pass

if all_vars_exist:
    print("\n✅ SUCCESS: All configuration variables are loaded correctly.")
    print("   You are clear to proceed with Data Unification and Training.")
else:
    print("\n❌ ERROR: Please re-run the 'Master Configuration Panel' (Cell 2) above to fix.")

--- Checking Required Variables for Training ---

✅ SUCCESS: All configuration variables are loaded correctly.
   You are clear to proceed with Data Unification and Training.


## 3. Download Dataset from Roboflow

This cell uses the configuration you provided above to download the correct dataset from Roboflow.

In [4]:
import roboflow
from google.colab import userdata
import os
import zipfile
import glob
import shutil
import yaml
import random

# --- HELPER FUNCTIONS ---
def to_xyxy(parts):
    xc, yc, w, h = parts[1], parts[2], parts[3], parts[4]
    return [parts[0], xc - w/2, yc - h/2, xc + w/2, yc + h/2]

def get_center(box):
    return ((box[1] + box[3]) / 2, (box[2] + box[4]) / 2)

def is_inside_center_point(inner_box, outer_box):
    inner_center = get_center(inner_box)
    return (outer_box[1] <= inner_center[0] <= outer_box[3] and
            outer_box[2] <= inner_center[1] <= outer_box[4])

def sanitize_labels(path):
    print(f"   🧹 Sanitizing labels in {os.path.basename(path)}...")
    for fp in glob.glob(os.path.join(path, "**", "*.txt"), recursive=True):
        if 'classes' in fp or 'README' in fp: continue
        try:
            with open(fp, 'r') as f: lines = f.readlines()
            new_lines = []
            changed = False
            for l in lines:
                parts = list(map(float, l.strip().split()))
                if len(parts) == 5: new_lines.append(l)
                elif len(parts) > 5:
                    changed = True; cls, coords = int(parts[0]), parts[1:]
                    xs, ys = coords[0::2], coords[1::2]
                    new_lines.append(f"{cls} {(min(xs)+max(xs))/2:.6f} {(min(ys)+max(ys))/2:.6f} {max(xs)-min(xs):.6f} {max(ys)-min(ys):.6f}\n")
            if changed:
                with open(fp, 'w') as f: f.writelines(new_lines)
        except: continue

def get_class_map(dataset_path):
    yaml_path = os.path.join(dataset_path, 'data.yaml')
    with open(yaml_path, 'r') as f: config = yaml.safe_load(f)
    names = config.get('names', []); return {n: i for i, n in enumerate(names)}

# --- CORRECTED "INTELLIGENT" SPATIAL FILTER ---
def filter_by_spatial_containment(pool_path, containers, contents):
    print(f"\n📐 Running Spatial Filter...")
    print(f"   Rule: EVERY instrument must contain AT LEAST ONE of {contents}")

    class_map = get_class_map(pool_path)
    container_ids = {class_map[n] for n in containers if n in class_map}
    content_ids = {class_map[n] for n in contents if n in class_map}

    if not container_ids or not content_ids:
        print("⚠️ Missing classes for filter. Skipping."); return

    deleted_count, kept_count = 0, 0

    for file_path in glob.glob(os.path.join(pool_path, "labels", "*.txt")):
        try:
            with open(file_path, 'r') as f: lines = f.readlines()

            all_boxes = [to_xyxy(list(map(float, l.strip().split()))) for l in lines if len(l.strip().split()) == 5]
            container_boxes = [b for b in all_boxes if int(b[0]) in container_ids]
            content_boxes = [b for b in all_boxes if int(b[0]) in content_ids]

            if not container_boxes:
                # No instruments on this image, so it doesn't violate the rule. Keep it.
                image_is_valid = True
            else:
                image_is_valid = True
                for container in container_boxes:
                    has_any_content = False
                    for content in content_boxes:
                        if is_inside_center_point(content, container):
                            has_any_content = True
                            break

                    if not has_any_content:
                        image_is_valid = False
                        break

            if image_is_valid:
                kept_count += 1
            else:
                deleted_count += 1
                os.remove(file_path)
                base = os.path.splitext(os.path.basename(file_path))[0]
                for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                    p = os.path.join(pool_path, "images", base + ext)
                    if os.path.exists(p): os.remove(p)
        except: continue

    print(f"✅ Spatial Filter Complete. Kept: {kept_count}, Deleted: {deleted_count}")

def unify_and_split_datasets(versions, output_path, ratios):
    # This function is unchanged
    pool_dir = "temp_pool"
    if os.path.exists(pool_dir): shutil.rmtree(pool_dir)
    os.makedirs(os.path.join(pool_dir, "images")); os.makedirs(os.path.join(pool_dir, "labels"))
    if os.path.exists(output_path): shutil.rmtree(output_path)
    for s in ['train', 'valid', 'test']: os.makedirs(os.path.join(output_path, s, 'images')); os.makedirs(os.path.join(output_path, s, 'labels'))
    print(f"\n🔄 Pooling {len(versions)} versions...")
    for i, v_path in enumerate(versions):
        if i == 0:
            yp = os.path.join(v_path, 'data.yaml')
            if os.path.exists(yp): shutil.copy(yp, os.path.join(pool_dir, 'data.yaml'))
        for sub in ['train', 'valid', 'test']:
            src_img, src_lbl = os.path.join(v_path, sub, 'images'), os.path.join(v_path, sub, 'labels')
            if not os.path.exists(src_img): continue
            for f in os.listdir(src_img):
                if f.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    new_name = f"v{i}_{sub}_{f}"; shutil.copy(os.path.join(src_img, f), os.path.join(pool_dir, "images", new_name))
                    lbl_name = os.path.splitext(f)[0] + ".txt"
                    if os.path.exists(os.path.join(src_lbl, lbl_name)):
                        shutil.copy(os.path.join(src_lbl, lbl_name), os.path.join(pool_dir, "labels", os.path.splitext(new_name)[0] + ".txt"))
    sanitize_labels(pool_dir)
    if REQUIRE_SPATIAL_CONTAINMENT:
        filter_by_spatial_containment(pool_dir, CONTAINER_CLASSES, REQUIRED_CONTENTS)
    all_images = glob.glob(os.path.join(pool_dir, "images", "*")); random.shuffle(all_images)
    count = len(all_images); n_train, n_val = int(count * ratios[0]), int(count * ratios[1])
    train_set, val_set, test_set = all_images[:n_train], all_images[n_train:n_train+n_val], all_images[n_train+n_val:]
    print(f"\n✂️ Splitting Dataset (Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)})")
    def move_set(file_list, split_name):
        for img_path in file_list:
            base = os.path.basename(img_path); shutil.move(img_path, os.path.join(output_path, split_name, "images", base))
            lbl_name = os.path.splitext(base)[0] + ".txt"
            if os.path.exists(os.path.join(pool_dir, "labels", lbl_name)):
                shutil.move(os.path.join(pool_dir, "labels", lbl_name), os.path.join(output_path, split_name, "labels", lbl_name))
    move_set(train_set, "train"); move_set(val_set, "valid"); move_set(test_set, "test")
    yp = os.path.join(pool_dir, "data.yaml")
    if os.path.exists(yp):
        with open(yp, 'r') as f: y = yaml.safe_load(f)
        y['path'], y['train'], y['val'], y['test'] = os.path.abspath(output_path), "train/images", "valid/images", "test/images"
        with open(os.path.join(output_path, "data.yaml"), 'w') as f: yaml.dump(y, f)
        return os.path.join(output_path, "data.yaml")
    return None

# --- MAIN EXECUTION ---
try:
    ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
    rf = roboflow.Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace(ROBOFLOW_WORKSPACE_ID).project(ROBOFLOW_PROJECT_ID)
    if isinstance(ROBOFLOW_VERSIONS, int): ROBOFLOW_VERSIONS = [ROBOFLOW_VERSIONS]
    d_paths = []
    for v in ROBOFLOW_VERSIONS:
        print(f"\n⬇️ Downloading V{v}..."); ds = project.version(v).download("yolov11")
        loc = ds.location;
        if loc.endswith('.zip'):
            with zipfile.ZipFile(loc, 'r') as z: z.extractall(os.path.splitext(loc)[0]); loc = os.path.splitext(loc)[0]
        d_paths.append(loc)
    FINAL_DIR = os.path.join(os.getcwd(), "merged_dataset")
    DATA_YAML_PATH = unify_and_split_datasets(d_paths, FINAL_DIR, [TRAIN_RATIO, VAL_RATIO, TEST_RATIO])
    if DATA_YAML_PATH and os.path.exists(DATA_YAML_PATH):
        print(f"\n✅ Dataset Ready!"); print(f"   Config: {DATA_YAML_PATH}")
    else:
        print("❌ Error generating dataset.")
except Exception as e:
    print(f"❌ Error: {e}")

loading Roboflow workspace...
loading Roboflow project...

⬇️ Downloading V24...



Extracting Dataset Version Zip to hvacai-24 in yolov11:: 100%|██████████| 15227/15227 [00:06<00:00, 2398.65it/s]



⬇️ Downloading V27...



Extracting Dataset Version Zip to hvacai-27 in yolov11:: 100%|██████████| 483/483 [00:00<00:00, 5379.65it/s]



⬇️ Downloading V28...



Extracting Dataset Version Zip to hvacai-28 in yolov11:: 100%|██████████| 14873/14873 [00:08<00:00, 1712.71it/s]



🔄 Pooling 3 versions...
   🧹 Sanitizing labels in temp_pool...

📐 Running Spatial Filter...
   Rule: EVERY instrument must contain AT LEAST ONE of ['id_letters', 'tag_number']
✅ Spatial Filter Complete. Kept: 7231, Deleted: 8044

✂️ Splitting Dataset (Train: 5784, Val: 1084, Test: 363)

✅ Dataset Ready!
   Config: /content/YOLOplan/merged_dataset/data.yaml


In [ ]:
#@title 🚀 Train The Model
import os
import glob

# --- 2. DISABLE EXTERNAL LOGGERS ---
# Ensures a clean, local-only run.
!yolo settings tensorboard=False wandb=False mlflow=False

# --- 3. DEFINE PATHS & DETERMINE STARTING WEIGHTS ---
current_run_dir = os.path.join(DRIVE_ROOT, RUN_NAME)
crash_ckpt = os.path.join(current_run_dir, "weights", "last.pt")
weights_to_load, should_resume = MODEL, False

if os.path.exists(crash_ckpt):
    print(f"🚨 CRASH DETECTED: Resuming from '{crash_ckpt}'")
    weights_to_load, should_resume = crash_ckpt, True
elif USE_PREVIOUS_WEIGHTS and os.path.exists(PREVIOUS_WEIGHTS_PATH):
    print(f"🧠 PROGRESSIVE MODE: Loading intelligence from '{PREVIOUS_WEIGHTS_PATH}'")
    weights_to_load = PREVIOUS_WEIGHTS_PATH
else:
    print(f"🆕 COLD START: Training from standard {MODEL}")

# --- 4. ENSURE DATA & BUILD COMMAND ---
if 'DATA_YAML_PATH' not in locals(): raise FileNotFoundError("❌ data.yaml not found. Run Cell 3 first.")
cmd_args = [
    f"task=detect",
    f"mode=train",
    f"model='{weights_to_load}'",
    f"data='{DATA_YAML_PATH}'",
    f"project='{DRIVE_ROOT}'",
    f"name='{RUN_NAME}'",
    f"exist_ok=True"
]

if not should_resume:
    cmd_args.extend([
        f"epochs={EPOCHS}",
        f"imgsz={IMG_SIZE}",
        f"batch={BATCH_SIZE}"
    ])
    if 'CLS_GAIN' in locals(): cmd_args.append(f"cls={CLS_GAIN}")

    if not ENABLE_AUGMENTATIONS:
        cmd_args.extend(["mosaic=0.0", "degrees=0.0", "translate=0.0", "scale=0.0", "shear=0.0",
                         "perspective=0.0", "flipud=0.0", "fliplr=0.0", "mixup=0.0",
                         "copy_paste=0.0", "hsv_h=0.0", "hsv_s=0.0", "hsv_v=0.0"])
else:
    cmd_args.append("resume=True")

final_cmd = "yolo " + " ".join(cmd_args)
print(f"\n🚀 COMMAND: {final_cmd}\n")

# --- 5. EXECUTE TRAINING ---
!{final_cmd}

print("\n\n✅ Training Complete!")
print(f"   Results, weights, and charts saved to: {current_run_dir}")

✅ Updated 'tensorboard=False'
✅ Updated 'wandb=False'
✅ Updated 'mlflow=False'
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/YOLOplan/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": false,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": false,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
🧠 PROGRESSIVE MODE: Loading intelligence from '/content/drive/MyDrive/hvac_training/yolo11m_run_v8/weights/best.pt'

🚀 COMMAND: yolo task=detect mode=train model='/content/drive/MyDrive/hvac_training/yolo11m_run_v8/weights/best.pt' data='/content/YOLOplan/merged_dataset/data.yaml' pro

## 5. Validate and Test

After training, these cells will help you validate the model's performance on the test set and run predictions on new images. The results will be saved in the project folder you defined.

In [ ]:
#@title 🔎 Find Best Model from Last Run
import os
import glob

# Construct the expected path using the variables from your Master Config
# This points to the results of your last training run.
model_path = os.path.join(DRIVE_ROOT, RUN_NAME, "weights", "best.pt")

print(f"Searching for trained model at: {model_path}")

if os.path.exists(model_path):
    BEST_WEIGHTS_PATH = model_path
    print(f"✅ Model Found: {BEST_WEIGHTS_PATH}")
else:
    BEST_WEIGHTS_PATH = None
    print(f"❌ Model Not Found! Please ensure a training run has completed and saved to the correct Drive folder.")

In [ ]:
#@title 🔮 Run Prediction & Visualize Results
import os
import glob
import random
from IPython.display import Image, display

# 1. CHECK IF MODEL WAS FOUND
if 'BEST_WEIGHTS_PATH' in locals() and BEST_WEIGHTS_PATH:

    # 2. FIND VALIDATION IMAGES
    if 'DATA_YAML_PATH' in locals() and os.path.exists(DATA_YAML_PATH):
        dataset_root = os.path.dirname(DATA_YAML_PATH)
        valid_images_path = os.path.join(dataset_root, "valid", "images")

        if os.path.exists(valid_images_path):

            # 3. RUN PREDICTION
            print(f"🔮 Running prediction with model: {BEST_WEIGHTS_PATH}")

            # Use the new Drive path for saving results
            output_dir = os.path.join(DRIVE_ROOT, f"{RUN_NAME}_predictions")

            # Run YOLO inference on the entire validation set
            !yolo task=detect mode=predict \
                model="{BEST_WEIGHTS_PATH}" \
                source="{valid_images_path}" \
                imgsz={IMG_SIZE} \
                conf=0.25 \
                project="{DRIVE_ROOT}" \
                name="{RUN_NAME}_predictions" \
                save=True \
                max_det=100 \
                exist_ok=True

            # 4. VISUALIZE RESULTS
            predicted_images = glob.glob(os.path.join(output_dir, '*.jpg'))

            if predicted_images:
                print(f"\n👀 Displaying 3 Random Sample Results:")
                display_samples = random.sample(predicted_images, min(len(predicted_images), 3))

                for img_path in display_samples:
                    print(f"--- Result: {os.path.basename(img_path)} ---")
                    display(Image(filename=img_path, width=800))
                    print("\n")
            else:
                print("⚠️ Prediction ran, but no output images were found.")
        else:
            print(f"❌ Validation image folder not found at: {valid_images_path}")
    else:
        print("❌ DATA_YAML_PATH not found. Ensure dataset is downloaded.")
else:
    print("❌ BEST_WEIGHTS_PATH not found. Run the 'Find Best Model' cell first.")

In [ ]:
import shutil
import os
from google.colab import files
from datetime import datetime

# Define the source folder containing all your runs
source_folder = '/content/drive/MyDrive/hvac_training/yolo11m_run_v4'

# Check if the folder exists before trying to zip
if os.path.exists(source_folder):
    # Create a unique filename with a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    zip_filename = f"YOLO11_Training_Results_{timestamp}"

    print(f"📦 Compressing '{source_folder}'...")

    # Create the zip file (shutil.make_archive adds .zip automatically)
    shutil.make_archive(zip_filename, 'zip', source_folder)

    output_file = f"{zip_filename}.zip"
    file_size_mb = os.path.getsize(output_file) / (1024 * 1024)

    print(f"✅ Compression complete. File size: {file_size_mb:.2f} MB")
    print(f"⬇️ Starting download of: {output_file}")

    # Trigger the browser download
    files.download(output_file)
else:
    print(f"❌ Folder not found: {source_folder}")
    print("Did you run the training cell successfully?")